# 🎯 Advanced Grasp Detection v6

## Features
| Feature | Description |
|---------|-------------|
| **🔍 YOLO Detection** | Object detection + Grid sizing |
| **🎯 Smart Grasp** | Auto grasp point selection |
| **📏 Precise Sizing** | Contour-based measurements |
| **🦾 Tight Grip** | No margin - grip exact size |

## 1️⃣ Imports

In [1]:
import sys
import cv2
import numpy as np
import time
import socket
import serial
import torch
from collections import deque
from ultralytics import YOLO

sys.path.append('Depth-Anything-V2')

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print("✓ Imports loaded")

PyTorch: 2.9.1+cpu
CUDA: False
✓ Imports loaded


## 2️⃣ Hardware Configuration

In [2]:
ROBOT_IP = '192.168.1.6'
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200
CAMERA_ID = 2

HOMOGRAPHY_MATRIX = np.array([
    [0.005703976266962427, -0.3265299161278153, 88.58634169557483],
    [-0.47704058225560797, 0.015355046930804153, 172.0941543570439],
    [-0.00029949919510557677, 0.00018728182448344945, 1.0],
], dtype=np.float32)

print("✓ Hardware config loaded")

✓ Hardware config loaded


---
# 🔧 CALIBRATION SECTION
---

## 📐 Calibration 1: PIXELS_PER_MM
1. วางไม้บรรทัดบนพื้นที่ทำงาน
2. ลากเส้น แล้วกด ENTER
3. ใส่ขนาดจริง (mm)
4. Copy ค่าไปใส่ Configuration

In [ ]:
# === PIXELS_PER_MM CALIBRATION ===
# Uncomment และ run เมื่อต้องการ calibrate

'''
drawing = False
start_pt = None
end_pt = None
px_dist = 0

def callback(event, x, y, flags, param):
    global drawing, start_pt, end_pt, px_dist
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_pt = (x, y)
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        end_pt = (x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        end_pt = (x, y)
        px_dist = np.sqrt((end_pt[0]-start_pt[0])**2 + (end_pt[1]-start_pt[1])**2)
        print(f"📏 Distance: {px_dist:.1f} px")

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Pixel Calibration')
cv2.setMouseCallback('Pixel Calibration', callback)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    if start_pt and end_pt:
        cv2.line(frame, start_pt, end_pt, (0,255,0), 2)
    cv2.imshow('Pixel Calibration', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == 13 and px_dist > 0:
        real_mm = float(input("📏 Real size (mm): "))
        ppm = px_dist / real_mm
        print(f"✅ PIXELS_PER_MM = {ppm:.4f}")
cap.release()
cv2.destroyAllWindows()
'''

## 📏 Calibration 2: DEPTH_Z_SCALE
Run หลังจาก Load Depth Model

## 🦾 Calibration 3: Gripper Test
Run หลังจาก Connect Gripper

---
## 3️⃣ Configuration (ใส่ค่าที่ Calibrate แล้ว)
---

In [3]:
# === CALIBRATED VALUES ===
PIXELS_PER_MM = 2.2167
DEPTH_Z_SCALE = 57.428993

# === Z Heights ===
Z_FLOOR = -64
Z_SAFE = -40
Z_APPROACH = -55

# === Drop Position ===
DROP_POS = (-253.07, 115.17, -17.07, -62.78)

# === Gripper (NO MARGIN) ===
GRIPPER_SERVO_OPEN_ANGLE = 22
GRIPPER_SERVO_CLOSE_ANGLE = 96
GRIPPER_MAX_WIDTH_MM = 54
GRIPPER_MIN_WIDTH_MM = 0
GRIPPER_OPEN_MARGIN_MM = 5
GRIPPER_GRIP_MARGIN_MM = 0  # คีบตรงตามขนาด

# === Detection ===
MIN_OBJECT_AREA = 1000
YOLO_CONFIDENCE = 0.25

# === Depth Model ===
DEPTH_MODEL_PATH = 'Depth-Anything-V2/checkpoints/depth_anything_v2_vits.pth'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# === Grid ===
SHOW_GRID = True
GRID_SIZE_MM = 20

print("✓ Configuration loaded")
print(f"  GRIP_MARGIN: {GRIPPER_GRIP_MARGIN_MM}mm (tight grip)")

✓ Configuration loaded
  GRIP_MARGIN: 0mm (tight grip)


## 4️⃣ Load Models

In [4]:
from depth_anything_v2.dpt import DepthAnythingV2

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
}

print("Loading DepthAnything V2...")
depth_model = DepthAnythingV2(**model_configs['vits'])
depth_model.load_state_dict(torch.load(DEPTH_MODEL_PATH, map_location='cpu'))
depth_model = depth_model.to(DEVICE).eval()
print(f"✅ Depth model on {DEVICE}")

print("Loading YOLOv8...")
yolo_model = YOLO('yolov8n.pt')
print("✅ YOLO loaded")

xFormers not available
xFormers not available


Loading DepthAnything V2...
✅ Depth model on cpu
Loading YOLOv8...
✅ YOLO loaded


## 5️⃣ Classes Definition

In [5]:
class SmartGripperController:
    CALIB_ANGLES = np.array([22, 30, 40, 50, 60, 70, 80, 90, 96])
    CALIB_WIDTHS = np.array([54.0, 52.0, 48.0, 40.0, 32.0, 23.0, 12.0, 3.0, 0.0])
    
    def __init__(self, port='COM9', baudrate=115200):
        self.port = port
        self.baudrate = baudrate
        self.serial = None
        self.target_width = None
        
    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudrate, timeout=2)
            time.sleep(2)
            print(f"✅ Gripper connected on {self.port}")
            return True
        except Exception as e:
            print(f"❌ Failed: {e}")
            return False
    
    def disconnect(self):
        if self.serial: self.serial.close()
    
    def send_command(self, cmd):
        if self.serial:
            self.serial.write((cmd + '\n').encode())
            time.sleep(0.3)
    
    def mm_to_angle(self, width_mm):
        width = max(0.0, min(54.0, width_mm))
        return int(round(np.interp(width, self.CALIB_WIDTHS[::-1], self.CALIB_ANGLES[::-1])))
    
    def open_for_object(self, width_mm):
        self.target_width = width_mm
        open_w = min(54.0, width_mm + GRIPPER_OPEN_MARGIN_MM)
        angle = self.mm_to_angle(open_w)
        print(f"🦾 Open: {width_mm:.1f}mm → {open_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def grip_object(self, width_mm):
        grip_w = max(0.0, width_mm - GRIPPER_GRIP_MARGIN_MM)
        angle = self.mm_to_angle(grip_w)
        print(f"🦾 Grip: {width_mm:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def release(self):
        open_w = min(54.0, (self.target_width or 30) + 10)
        self.send_command(f'G{self.mm_to_angle(open_w)}')
        self.target_width = None


class DobotControllerTCP:
    def __init__(self, homography_matrix=None):
        self.dashboard_port = 29999
        self.sock = None
        self.H = homography_matrix
        
    def connect(self, ip):
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(5)
            self.sock.connect((ip, self.dashboard_port))
            self.send_command("ClearError()")
            time.sleep(0.5)
            self.send_command("EnableRobot()")
            time.sleep(4)
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            print("✅ Robot connected!")
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock:
            self.sock.send((cmd + "\n").encode("utf-8"))
            return self.sock.recv(1024).decode("utf-8")

    def home(self):
        print("🤖 HOME...")
        self.send_command("MovJ(-253.07, 115.17, -17.07, -62.78)")
        time.sleep(4)

    def move_to(self, x, y, z, r=0):
        cmd = f"MovJ({x},{y},{z},{r})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def move_to_and_wait(self, x, y, z, r=0, wait=3):
        self.move_to(x, y, z, r)
        time.sleep(wait)
    
    def joint_move(self, j1=0, j2=0, j3=0, j4=0):
        cmd = f"JointMovJ({j1},{j2},{j3},{j4})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def joint_move_and_wait(self, j1=0, j2=0, j3=0, j4=0, wait=3):
        self.joint_move(j1, j2, j3, j4)
        time.sleep(wait)

    def pixel_to_robot(self, u, v):
        if self.H is None: return None, None
        pt = np.array([u, v, 1], dtype=np.float32)
        res = np.dot(self.H, pt)
        return res[0]/res[2], res[1]/res[2]

print("✓ Gripper & Robot classes loaded")

✓ Gripper & Robot classes loaded


In [6]:
class PreciseSizeDetector:
    """YOLO + Contour + Grid"""
    
    def __init__(self, yolo_model, pixels_per_mm):
        self.yolo = yolo_model
        self.ppm = pixels_per_mm
        self.bg_gray = None
    
    def set_background(self, frame):
        self.bg_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        self.bg_gray = cv2.GaussianBlur(self.bg_gray, (5,5), 0)
        print("✅ Background set")
    
    def detect(self, frame):
        objects = []
        results = self.yolo(frame, conf=YOLO_CONFIDENCE, verbose=False)
        
        for r in results:
            for box in r.boxes:
                x1,y1,x2,y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                
                roi = frame[y1:y2, x1:x2]
                if roi.size == 0: continue
                
                gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if contours:
                    cnt = max(contours, key=cv2.contourArea)
                    cnt += np.array([x1, y1])
                    rect = cv2.minAreaRect(cnt)
                    cx, cy = int(rect[0][0]), int(rect[0][1])
                    
                    objects.append({
                        'bbox': (x1, y1, x2-x1, y2-y1),
                        'center': (cx, cy),
                        'rect': rect,
                        'rect_size': rect[1],
                        'rect_angle': rect[2],
                        'contour': cnt,
                        'conf': conf,
                        'area': cv2.contourArea(cnt)
                    })
        
        if not objects:
            objects = self.edge_detect(frame)
        
        return objects
    
    def edge_detect(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5,5), 0)
        edges = cv2.Canny(blur, 50, 150)
        edges = cv2.dilate(edges, np.ones((3,3), np.uint8), iterations=2)
        
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        objects = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > MIN_OBJECT_AREA:
                hull = cv2.convexHull(cnt)
                rect = cv2.minAreaRect(hull)
                x,y,w,h = cv2.boundingRect(hull)
                
                objects.append({
                    'bbox': (x,y,w,h),
                    'center': (x+w//2, y+h//2),
                    'rect': rect,
                    'rect_size': rect[1],
                    'rect_angle': rect[2],
                    'contour': hull,
                    'area': area
                })
        
        return sorted(objects, key=lambda o: o['area'], reverse=True)
    
    def draw_grid(self, frame):
        if not SHOW_GRID: return frame
        h, w = frame.shape[:2]
        grid_px = int(GRID_SIZE_MM * self.ppm)
        for x in range(0, w, grid_px):
            cv2.line(frame, (x,0), (x,h), (50,50,50), 1)
        for y in range(0, h, grid_px):
            cv2.line(frame, (0,y), (w,y), (50,50,50), 1)
        cv2.putText(frame, f"Grid: {GRID_SIZE_MM}mm", (10, h-10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,100,100), 1)
        return frame

print("✓ Detector class loaded")

✓ Detector class loaded


In [7]:
class SmartGraspSelector:
    """Auto select best grasp + alternatives"""
    
    def __init__(self, pixels_per_mm):
        self.ppm = pixels_per_mm
    
    def analyze_object(self, obj):
        w, h = obj['rect_size']
        angle = obj['rect_angle']
        cx, cy = obj['center']
        
        aspect = max(w,h) / (min(w,h) + 0.001)
        is_ring = self.detect_ring(obj)
        
        grasps = []
        
        if aspect > 2.0:
            # Long object → narrow side
            if w < h:
                grip_w = w / self.ppm
                grip_angle = angle + 90
            else:
                grip_w = h / self.ppm
                grip_angle = angle
            
            grasps.append({
                'center': (cx, cy),
                'width_mm': grip_w,
                'angle': self.normalize_angle(grip_angle),
                'score': 1.0,
                'type': 'narrow_side'
            })
            
            alt_angle = grip_angle + 90
            alt_w = max(w,h) / self.ppm
            if alt_w <= GRIPPER_MAX_WIDTH_MM:
                grasps.append({
                    'center': (cx, cy),
                    'width_mm': alt_w,
                    'angle': self.normalize_angle(alt_angle),
                    'score': 0.5,
                    'type': 'alternative'
                })
        
        elif is_ring:
            inner_w = self.estimate_ring_width(obj)
            for i in range(4):
                grasps.append({
                    'center': (cx, cy),
                    'width_mm': inner_w,
                    'angle': i * 45,
                    'score': 1.0 if i == 0 else 0.8,
                    'type': 'ring'
                })
        
        else:
            if w < h:
                grip_w = w / self.ppm
                grip_angle = angle + 90
            else:
                grip_w = h / self.ppm
                grip_angle = angle
            
            grasps.append({
                'center': (cx, cy),
                'width_mm': grip_w,
                'angle': self.normalize_angle(grip_angle),
                'score': 1.0,
                'type': 'default'
            })
            
            perp_w = max(w,h) / self.ppm
            if perp_w <= GRIPPER_MAX_WIDTH_MM:
                grasps.append({
                    'center': (cx, cy),
                    'width_mm': perp_w,
                    'angle': self.normalize_angle(grip_angle + 90),
                    'score': 0.6,
                    'type': 'perpendicular'
                })
        
        return grasps
    
    def detect_ring(self, obj):
        cnt = obj.get('contour')
        if cnt is None: return False
        area = cv2.contourArea(cnt)
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        return (area / (hull_area + 0.001)) < 0.7
    
    def estimate_ring_width(self, obj):
        w, h = obj['rect_size']
        return max(w, h) / self.ppm * 0.3
    
    def normalize_angle(self, angle):
        while angle > 90: angle -= 180
        while angle < -90: angle += 180
        return angle

print("✓ Grasp selector loaded")

✓ Grasp selector loaded


In [8]:
class RobustDepthEstimator:
    """Multi-sample + temporal averaging"""
    
    def __init__(self, model, device='cpu', history_size=5):
        self.model = model
        self.device = device
        self.floor_depth = None
        self.history = deque(maxlen=history_size)
    
    def estimate_depth(self, frame):
        return self.model.infer_image(frame)
    
    def calibrate_floor(self, frame):
        depth = self.estimate_depth(frame)
        h, w = depth.shape
        center = depth[h//3:2*h//3, w//3:2*w//3]
        self.floor_depth = np.median(center)
        print(f"✅ Floor depth: {self.floor_depth:.4f}")
        return self.floor_depth
    
    def get_object_height(self, depth_map, obj, scale):
        if self.floor_depth is None: return 0
        
        x, y, w, h = obj['bbox']
        region = depth_map[y:y+h, x:x+w]
        if region.size == 0: return 0
        
        samples = [np.median(region)]
        qh, qw = h//4, w//4
        if qh > 0 and qw > 0:
            samples.extend([
                np.median(region[:qh, :qw]),
                np.median(region[:qh, -qw:]),
                np.median(region[-qh:, :qw]),
                np.median(region[-qh:, -qw:])
            ])
        
        obj_depth = np.median(samples)
        height = max(0, (obj_depth - self.floor_depth) * scale)
        
        self.history.append(height)
        return np.median(self.history)
    
    def calculate_z(self, height_mm):
        z = Z_FLOOR + (height_mm * 0.5)
        return max(Z_FLOOR, min(Z_SAFE, z))

print("✓ Depth estimator loaded")

✓ Depth estimator loaded


## 6️⃣ Initialize & Connect

In [9]:
gripper = SmartGripperController(port=ESP32_PORT, baudrate=ESP32_BAUDRATE)
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX)
detector = PreciseSizeDetector(yolo_model, PIXELS_PER_MM)
grasp_selector = SmartGraspSelector(PIXELS_PER_MM)
depth_estimator = RobustDepthEstimator(depth_model, device=DEVICE)
print("✓ Components initialized")

✓ Components initialized


In [11]:
gripper.connect()

✅ Gripper connected on COM9


True

In [12]:
robot.connect(ROBOT_IP)

✅ Robot connected!


True

## 📷 Capture Background (Optional)

In [17]:
print("📷 BACKGROUND CALIBRATION")
print("SPACE = Capture | Q = Skip")

cap = cv2.VideoCapture(CAMERA_ID)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    cv2.putText(frame, "SPACE=Capture | Q=Skip", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
    cv2.imshow('Background', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):
        detector.set_background(frame)
        depth_estimator.calibrate_floor(frame)
        break
    elif key == ord('q'):
        print("Skipped")
        break
cap.release()
cv2.destroyAllWindows()

📷 BACKGROUND CALIBRATION
SPACE = Capture | Q = Skip
✅ Background set
✅ Floor depth: 3.0468


---
# 🎯 MAIN PICK-AND-PLACE
---

In [18]:
# =============================================================================
# 🎯 PICK-AND-PLACE v6
# =============================================================================
selected_object = None
selected_grasp = None
detected_objects = []
current_grasps = []
current_depth = None

def mouse_callback(event, x, y, flags, param):
    global selected_object, selected_grasp, current_grasps
    
    if event == cv2.EVENT_LBUTTONDOWN:
        for g in current_grasps:
            gx, gy = g['center']
            if abs(x - gx) < 20 and abs(y - gy) < 20:
                selected_grasp = g
                print(f"\n🎯 Grasp: W={g['width_mm']:.1f}mm A={g['angle']:.1f}° ({g['type']})")
                return
        
        for obj in detected_objects:
            bx, by, bw, bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                selected_object = obj
                current_grasps = grasp_selector.analyze_object(obj)
                selected_grasp = current_grasps[0] if current_grasps else None
                if selected_grasp:
                    print(f"\n📦 Object: {len(current_grasps)} grasps")
                    print(f"   Best: W={selected_grasp['width_mm']:.1f}mm A={selected_grasp['angle']:.1f}°")
                break

def draw_grasps(frame, grasps, selected):
    for g in grasps:
        cx, cy = g['center']
        angle = g['angle']
        is_sel = (selected and g == selected)
        
        if is_sel:
            color = (0, 0, 255)
            thick = 3
        elif g['score'] >= 1.0:
            color = (0, 255, 0)
            thick = 2
        else:
            color = (0, 255, 255)
            thick = 1
        
        dx = int(30 * np.cos(np.radians(angle)))
        dy = int(30 * np.sin(np.radians(angle)))
        cv2.line(frame, (cx-dx, cy-dy), (cx+dx, cy+dy), color, thick)
        cv2.circle(frame, (cx, cy), 5, color, -1)

def pick_with_grasp(obj, grasp):
    cx, cy = grasp['center']
    grip_w = grasp['width_mm']
    robot_r = -grasp['angle']
    
    robot_x, robot_y = robot.pixel_to_robot(cx, cy)
    height = depth_estimator.get_object_height(current_depth, obj, DEPTH_Z_SCALE) if current_depth is not None else 0
    z_grasp = depth_estimator.calculate_z(height)
    
    print(f"\n🤖 Pick: W={grip_w:.1f}mm R={robot_r:.1f}° Z={z_grasp:.1f}")
    
    print("🔄 Safe position...")
    robot.joint_move_and_wait(0, 0, 0, 0, 3)
    
    gripper.open_for_object(grip_w)
    time.sleep(1)
    
    robot.move_to_and_wait(robot_x, robot_y, Z_APPROACH, robot_r, 3)
    robot.move_to_and_wait(robot_x, robot_y, z_grasp, robot_r, 2)
    
    gripper.grip_object(grip_w)
    time.sleep(1.5)
    
    robot.move_to_and_wait(robot_x, robot_y, Z_SAFE, robot_r, 2)
    robot.move_to_and_wait(*DROP_POS[:3], DROP_POS[3], 3)
    
    gripper.release()
    time.sleep(1)
    robot.home()
    print("✅ Complete!")

# Main loop
cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Pick v6')
cv2.setMouseCallback('Pick v6', mouse_callback)

frame_count = 0
print("="*50)
print("🎯 PICK v6")
print("Click=Select | SPACE=Execute | G=Grid | Q=Quit")
print("="*50)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    frame = detector.draw_grid(frame)
    
    frame_count += 1
    if frame_count % 10 == 0:
        current_depth = depth_estimator.estimate_depth(frame)
    
    detected_objects = detector.detect(frame)
    
    for obj in detected_objects:
        x, y, w, h = obj['bbox']
        is_sel = (selected_object and obj['center'] == selected_object['center'])
        color = (0, 0, 255) if is_sel else (0, 255, 0)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        if 'rect' in obj:
            box = cv2.boxPoints(obj['rect'])
            cv2.drawContours(frame, [np.int32(box)], 0, color, 1)
        
        rect_w, rect_h = obj['rect_size']
        w_mm = min(rect_w, rect_h) / PIXELS_PER_MM
        cv2.putText(frame, f"W:{w_mm:.0f}mm", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    if selected_object and current_grasps:
        draw_grasps(frame, current_grasps, selected_grasp)
    
    cv2.rectangle(frame, (0, 0), (640, 35), (30, 30, 30), -1)
    cv2.putText(frame, f"Objects:{len(detected_objects)} | Click | SPACE=Pick | G=Grid | Q=Quit",
               (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
    
    if selected_grasp:
        cv2.putText(frame, f"[GRASP: W={selected_grasp['width_mm']:.1f}mm - SPACE]",
                   (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    cv2.imshow('Pick v6', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        selected_object = None
        selected_grasp = None
        current_grasps = []
    elif key == ord('g'):
        SHOW_GRID = not SHOW_GRID
    elif key == ord('h'):
        robot.home()
    elif key == ord(' ') and selected_object and selected_grasp:
        pick_with_grasp(selected_object, selected_grasp)
        selected_object = None
        selected_grasp = None
        current_grasps = []

cap.release()
cv2.destroyAllWindows()

🎯 PICK v6
Click=Select | SPACE=Execute | G=Grid | Q=Quit

📦 Object: 1 grasps
   Best: W=206.6mm A=90.0°

📦 Object: 1 grasps
   Best: W=206.6mm A=90.0°

📦 Object: 1 grasps
   Best: W=206.6mm A=90.0°

📦 Object: 1 grasps
   Best: W=206.6mm A=90.0°

📦 Object: 1 grasps
   Best: W=206.6mm A=90.0°

📦 Object: 1 grasps
   Best: W=206.6mm A=90.0°

📦 Object: 1 grasps
   Best: W=206.6mm A=90.0°

📦 Object: 1 grasps
   Best: W=206.6mm A=90.0°

🤖 Pick: W=206.6mm R=-90.0° Z=-64.0
🔄 Safe position...
   → JointMovJ(0,0,0,0)
🦾 Open: 206.6mm → 54.0mm (22°)
   → MovJ(11.467768669128418,68.15058135986328,-55,-90.0)
   → MovJ(11.467768669128418,68.15058135986328,-64,-90.0)
🦾 Grip: 206.6mm (22°)
   → MovJ(11.467768669128418,68.15058135986328,-40,-90.0)
   → MovJ(-253.07,115.17,-17.07,-62.78)
🤖 HOME...
✅ Complete!

📦 Object: 1 grasps
   Best: W=206.6mm A=90.0°

🤖 Pick: W=206.6mm R=-90.0° Z=-64.0
🔄 Safe position...
   → JointMovJ(0,0,0,0)
🦾 Open: 206.6mm → 54.0mm (22°)
   → MovJ(11.467768669128418,68.1505813598

In [19]:
robot.home()

🤖 HOME...


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [ ]:
gripper.disconnect()
print("✅ Done")